In [30]:
import numpy as np
import pandas as pd
import scipy
import sqlite3
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report

from xgboost import XGBClassifier

In [31]:
print(os.listdir('wids_datathon_2025/TRAIN'))


['TRAIN_QUANTITATIVE_METADATA.xlsx', 'TRAINING_SOLUTIONS.xlsx', 'TRAIN_CATEGORICAL_METADATA.xlsx', 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv']


In [32]:
train_quant = pd.read_excel('wids_datathon_2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx')
train_cat = pd.read_excel('wids_datathon_2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx')
train_func_connectome = pd.read_csv('wids_datathon_2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')

cat_imputer = SimpleImputer(strategy='most_frequent') #column is cat data
train_cat[['PreInt_Demos_Fam_Child_Ethnicity']] = cat_imputer.fit_transform(train_cat[['PreInt_Demos_Fam_Child_Ethnicity']])

quant_imputer = SimpleImputer(strategy='median') #age col
train_quant[['MRI_Track_Age_at_Scan']] = quant_imputer.fit_transform(train_quant[['MRI_Track_Age_at_Scan']])

train_data = pd.merge(train_quant, train_cat, on='participant_id')

print(train_data.shape)
train_data.head()

(1213, 28)


,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,...,MRI_Track_Age_at_Scan,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,...,14.274127,2019,4,1.0,0.0,3.0,21.0,45.0,NaN,NaN
1,00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,...,10.739219,2017,1,0.0,9.0,2.0,21.0,0.0,21.0,45.0
2,04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,...,13.463381,2017,1,1.0,2.0,2.0,9.0,0.0,NaN,NaN
3,05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,...,9.572553,2018,1,3.0,8.0,2.0,18.0,10.0,18.0,0.0
4,06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,...,6.654574,2018,1,0.0,1.0,2.0,12.0,0.0,NaN,NaN


In [33]:
rescale = StandardScaler()
scale_func_train = rescale.fit_transform(train_func_connectome.drop(columns='participant_id'))

pca_med = PCA(n_components=0.95, random_state=40)
train_func_connectome_pca = pca_med.fit_transform(scale_func_train)

# convert reduced pca ver back to df
train_func_connectome_pca = pd.DataFrame(train_func_connectome_pca, columns = [f'PCA_{i}' for i in range(train_func_connectome_pca.shape[1])])
train_func_connectome_pca['participant_id'] = train_func_connectome['participant_id'].values

In [34]:
test_quant = pd.read_excel('wids_datathon_2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx')
test_cat = pd.read_excel('wids_datathon_2025/TEST/TEST_CATEGORICAL.xlsx')
test_func_connectome = pd.read_csv('wids_datathon_2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

scale_func_test = rescale.transform(test_func_connectome.drop(columns='participant_id'))

test_func_connectome_pca = pca_med.transform(scale_func_test)

# convert reduced pca ver back to df
test_func_connectome_pca = pd.DataFrame(test_func_connectome_pca, columns = [f'PCA_{i}' for i in range(test_func_connectome_pca.shape[1])])
test_func_connectome_pca['participant_id'] = test_func_connectome['participant_id'].values

test_data = pd.merge(test_quant, test_cat, on = 'participant_id')
test_data = test_data.merge(test_func_connectome_pca, on = 'participant_id')

test_data = test_data.drop(columns = ['participant_id'])

print(train_data.shape)
print(test_data.shape)

(1213, 28)
(304, 936)


In [35]:
train_targets = pd.read_excel('wids_datathon_2025/TRAIN/TRAINING_SOLUTIONS.xlsx')
train_targets.head()
#print(targets.shape)

,participant_id,ADHD_Outcome,Sex_F
0,UmrK0vMLopoR,1,1
1,CPaeQkhcjg7d,1,0
2,Nb4EetVPm3gs,1,0
3,p4vPhVu91o4b,1,1
4,M09PXs7arQ5E,1,1


In [36]:
final_data = pd.merge(train_data, train_targets, on = 'participant_id')

sex = final_data.pop('Sex_F')
adhd_outcome = final_data.pop('ADHD_Outcome')
final_data.insert(1, 'Sex_F', sex)
final_data.insert(1, 'ADHD_Outcome', adhd_outcome)

final_train_data = final_data.merge(train_func_connectome_pca, on='participant_id')

final_data.head()

,participant_id,ADHD_Outcome,Sex_F,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,...,MRI_Track_Age_at_Scan,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,00aIpNTbG5uh,1,0,100.00,13.0,3.0,15.0,44.0,14.0,20.0,...,14.274127,2019,4,1.0,0.0,3.0,21.0,45.0,NaN,NaN
1,00fV0OyyoLfw,1,0,92.27,14.0,3.0,12.0,35.0,25.0,28.0,...,10.739219,2017,1,0.0,9.0,2.0,21.0,0.0,21.0,45.0
2,04X1eiS79T4B,0,1,86.67,14.0,3.0,21.0,37.0,18.0,26.0,...,13.463381,2017,1,1.0,2.0,2.0,9.0,0.0,NaN,NaN
3,05ocQutkURd6,0,1,93.34,14.0,3.0,11.0,42.0,15.0,20.0,...,9.572553,2018,1,3.0,8.0,2.0,18.0,10.0,18.0,0.0
4,06YUNBA9ZRLq,1,0,0.00,14.0,8.0,12.0,35.0,22.0,12.0,...,6.654574,2018,1,0.0,1.0,2.0,12.0,0.0,NaN,NaN


In [37]:
X = final_train_data.drop(columns = ['ADHD_Outcome', 'Sex_F', 'participant_id'])
y = final_train_data[['ADHD_Outcome', 'Sex_F']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state= 40)

model_adhd = RandomForestClassifier(n_estimators=50, random_state=40, class_weight='balanced', max_features='sqrt')
model_sex = RandomForestClassifier(n_estimators=50, random_state=40)

#num_male = (y_train['Sex_F'] == 0).sum()  # Count instances of "Male" (0)
#num_female = (y_train['Sex_F'] == 1).sum()  # Count instances of "Female" (1)

#scale_pos_weight = num_male / num_female  # Ensure this is a float

#model_sex = XGBClassifier(scale_pos_weight=scale_pos_weight)

model_adhd.fit(X_train, y_train['ADHD_Outcome'])
model_sex.fit(X_train, y_train['Sex_F'])

y_pred_adhd = model_adhd.predict(X_val)
y_pred_sex = model_sex.predict(X_val)

# ADHD pred
print("RF ADHD_Outcome Accuracy:", accuracy_score(y_val['ADHD_Outcome'], y_pred_adhd))
#print(classification_report(y_val['ADHD_Outcome'], y_pred_adhd))

# Sex pred
print("RF Sex_F Accuracy:", accuracy_score(y_val['Sex_F'], y_pred_sex))
#print(classification_report(y_val['Sex_F'], y_pred_sex))


RF ADHD_Outcome Accuracy: 0.7225274725274725
RF Sex_F Accuracy: 0.6593406593406593


In [38]:
print("Sex_F Class Distribution in Training Data:")
print(y_train['Sex_F'].value_counts(normalize=True) * 100)


Sex_F Class Distribution in Training Data:
Sex_F
0    65.135453
1    34.864547
Name: proportion, dtype: float64


In [39]:
test_targets = pd.read_excel('wids_datathon_2025/SAMPLE_SUBMISSION.xlsx')
print(test_targets.shape)
print(final_data.shape)

X_test = test_data
y_test = test_targets[['ADHD_Outcome', 'Sex_F']]

print(set(X_train.columns) - set(X_test.columns))  # Features in training set but missing in test set
print(set(X_test.columns) - set(X_train.columns))  # Extra features in test set
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(304, 3)
(1213, 30)
set()
set()
(849, 936) (304, 936)
(849, 2) (304, 2)


In [40]:
y_test_preds_adhd = model_adhd.predict(X_test)
y_test_preds_sex = model_sex.predict(X_test)

adhd_accuracy = accuracy_score(y_test['ADHD_Outcome'], y_test_preds_adhd)
sex_accuracy = accuracy_score(y_test['Sex_F'], y_test_preds_sex)


print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_train sample: \n{y_train[:5]}")

print(f"y_test shape: {y_test.shape}")
print(f"y_pred_sex shape: {y_test_preds_sex.shape}")
print(f"y_pred_adhd shape: {y_test_preds_adhd.shape}")


print(f"ADHD Accuracy: {adhd_accuracy}")
print(f"Sex Accuracy: {sex_accuracy}")

X_train shape: (849, 936)
y_train shape: (849, 2)
y_train sample: 
      ADHD_Outcome  Sex_F
100              0      1
1076             1      0
339              0      1
387              0      1
759              0      0
y_test shape: (304, 2)
y_pred_sex shape: (304,)
y_pred_adhd shape: (304,)
ADHD Accuracy: 0.5460526315789473
Sex Accuracy: 0.4901315789473684


In [41]:
importances = model_adhd.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df.head(20))



                        Feature  Importance
11        SDQ_SDQ_Externalizing    0.045261
13        SDQ_SDQ_Hyperactivity    0.032345
12    SDQ_SDQ_Generating_Impact    0.026531
9    SDQ_SDQ_Difficulties_Total    0.018849
10   SDQ_SDQ_Emotional_Problems    0.007473
8      SDQ_SDQ_Conduct_Problems    0.006904
15        SDQ_SDQ_Peer_Problems    0.004619
618                     PCA_591    0.003987
14        SDQ_SDQ_Internalizing    0.003908
115                      PCA_88    0.003828
518                     PCA_491    0.003573
761                     PCA_734    0.003443
780                     PCA_753    0.003269
471                     PCA_444    0.003259
481                     PCA_454    0.003050
476                     PCA_449    0.002932
66                       PCA_39    0.002919
301                     PCA_274    0.002915
559                     PCA_532    0.002794
266                     PCA_239    0.002766


In [42]:
# Define weights (2x for ADHD_Outcome=1 and Sex_F=1)
weights = (y_test['ADHD_Outcome'] == 1) & (y_test['Sex_F'] == 1)
weights = weights.astype(int) * 2 + 1  # ADHD females get weight 2, others get 1

f1_adhd = f1_score(y_test['ADHD_Outcome'], y_test_preds_adhd, average="binary", sample_weight=weights)
f1_sex = f1_score(y_test['Sex_F'], y_test_preds_sex, average="binary")

# Compute final leaderboard score (average of the two)
final_score = (f1_adhd + f1_sex) / 2

print(f"Weighted F1 Score (ADHD): {f1_adhd:.4f}")
print(f"F1 Score (Sex): {f1_sex:.4f}")
print(f"Final Kaggle Leaderboard Score Estimate: {final_score:.4f}")

print("True ADHD Cases in y_test:", sum(y_test['ADHD_Outcome'] == 1))
print("Predicted ADHD Cases:", sum(y_test_preds_adhd == 1))


adhd_precision = precision_score(y_test['ADHD_Outcome'], y_test_preds_adhd, zero_division=1)
adhd_recall = recall_score(y_test['ADHD_Outcome'], y_test_preds_adhd)

print(f"ADHD Precision: {adhd_precision:.4f}")
print(f"ADHD Recall: {adhd_recall:.4f}")

sex_precision = precision_score(y_val['Sex_F'], y_pred_sex)
sex_recall = recall_score(y_val['Sex_F'], y_pred_sex)

print(f"Sex_F Precision: {sex_precision:.4f}")
print(f"Sex_F Recall: {sex_recall:.4f}")



Weighted F1 Score (ADHD): 0.7289
F1 Score (Sex): 0.0000
Final Kaggle Leaderboard Score Estimate: 0.3645
True ADHD Cases in y_test: 166
Predicted ADHD Cases: 228
ADHD Precision: 0.5614
ADHD Recall: 0.7711
Sex_F Precision: 0.0000
Sex_F Recall: 0.0000


In [43]:
print("Sex Classification Report:")
print(classification_report(y_val['Sex_F'], y_pred_sex))
print("ADHD Classification Report:")
print(classification_report(y_val['ADHD_Outcome'], y_pred_adhd))

Sex Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.98      0.79       244
           1       0.00      0.00      0.00       120

    accuracy                           0.66       364
   macro avg       0.33      0.49      0.40       364
weighted avg       0.45      0.66      0.53       364

ADHD Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.15      0.26       117
           1       0.71      0.99      0.83       247

    accuracy                           0.72       364
   macro avg       0.81      0.57      0.55       364
weighted avg       0.77      0.72      0.65       364



In [44]:
predictions = pd.DataFrame({
    'participant_id': test_targets['participant_id'],
    'ADHD_Outcome': y_test_preds_adhd,
    'Sex_F': y_test_preds_sex  })
predictions.to_csv('predictions_V3.csv', index = False)
predictions.head(10)

,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,1,0
1,vhGrzmvA3Hjq,1,0
2,ULliyEXjy4OV,1,0
3,LZfeAb1xMtql,1,0
4,EnFOUv0YK1RG,1,0
5,3VbkvJ22j9Fu,1,0
6,PRKZcnOgqcuk,0,0
7,DuVUuyMZi5qV,0,0
8,uM4etVLZrgMg,0,0
9,BpzyExrET5ta,1,0
